In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib

import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import PowerTransformer
from prettytable import PrettyTable
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler, LabelEncoder

In [ ]:
from tqdm import tqdm
import tensorflow as tf

from sklearn.metrics import average_precision_score, roc_auc_score, confusion_matrix

from scipy.special import softmax


In [ ]:
import gc

In [ ]:
!pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 6.2 MB/s eta 0:00:00


In [ ]:
from tensorflow_addons.activations import sparsemax

In [ ]:
!pip install torch pytorch-tabnet shap

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pytorch_tabnet.tab_model import TabNetClassifier
import shap

In [ ]:
np.random.seed(42)

In [ ]:
final_x_df = pd.read_pickle("/content/drive/MyDrive/Datasets/dataset/final_x_train.pkl")
final_y_df = pd.read_pickle("/content/drive/MyDrive/Datasets/dataset/final_y_train.pkl")
final_test_df = pd.read_pickle("/content/drive/MyDrive/Datasets/dataset/final_test_main.pkl")

In [ ]:
final_x_df

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix
0,2987000,68.500000,W,13926,NaN,150.0,discover,142.0,credit,315.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6,6,6,6
1,2987001,29.000000,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4,0,6,6
2,2987002,59.000000,W,4663,490.0,150.0,visa,166.0,debit,330.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5,0,6,6
3,2987003,50.000000,W,18132,567.0,150.0,mastercard,117.0,debit,476.0,...,NaN,NaN,NaN,NaN,NaN,NaN,9,0,6,6
4,2987004,50.000000,H,4497,514.0,150.0,mastercard,102.0,credit,420.0,...,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M,4,0,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,49.000000,W,6550,NaN,150.0,visa,226.0,debit,272.0,...,NaN,NaN,NaN,NaN,NaN,NaN,6,6,6,6
590536,3577536,39.500000,W,10444,225.0,150.0,mastercard,224.0,debit,204.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4,0,6,6
590537,3577537,30.953125,W,12037,595.0,150.0,mastercard,224.0,debit,231.0,...,NaN,NaN,NaN,NaN,NaN,NaN,4,0,6,6
590538,3577538,117.000000,W,7826,481.0,150.0,mastercard,224.0,debit,387.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,6,6


In [ ]:
categorical_columns = final_x_df.select_dtypes(include='category').columns

numerical_columns = final_x_df.select_dtypes(exclude='category').columns

In [ ]:
categorical_columns

Index(['ProductCD', 'card4', 'card6', 'P_emaildomain', 'R_emaildomain', 'M1',
       'M2', 'M3', 'M4', 'M5', 'M6', 'M7', 'M8', 'M9', 'id_12', 'id_15',
       'id_16', 'id_23', 'id_27', 'id_28', 'id_29', 'id_30', 'id_31', 'id_33',
       'id_34', 'id_35', 'id_36', 'id_37', 'id_38', 'DeviceType',
       'DeviceInfo'],
      dtype='object')

In [ ]:
numerical_columns

Index(['TransactionID', 'TransactionAmt', 'card1', 'card2', 'card3', 'card5',
       'addr1', 'addr2', 'dist1', 'dist2',
       ...
       'id_21', 'id_22', 'id_24', 'id_25', 'id_26', 'id_32',
       'P_emaildomain_bin', 'P_emaildomain_suffix', 'R_emaildomain_bin',
       'R_emaildomain_suffix'],
      dtype='object', length=405)

In [ ]:
scaler = MinMaxScaler()
final_x_df[numerical_columns] = scaler.fit_transform(final_x_df[numerical_columns])

# Encode categorical columns using LabelEncoder
label_encoder = LabelEncoder()
for column in categorical_columns:
    final_x_df[column] = label_encoder.fit_transform(final_x_df[column])

final_x_df.head()

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix
0,0.000000,0.002137,4,0.743044,NaN,0.381679,2,0.306569,2,0.488636,...,2,2,2,2,2,1786,0.666667,0.75,0.666667,0.75
1,0.000002,0.000900,4,0.100885,0.608,0.381679,3,0.014599,2,0.511364,...,2,2,2,2,2,1786,0.444444,0.00,0.666667,0.75
2,0.000003,0.001840,4,0.210566,0.780,0.381679,4,0.481752,3,0.522727,...,2,2,2,2,2,1786,0.555556,0.00,0.666667,0.75
3,0.000005,0.001558,4,0.984824,0.934,0.381679,3,0.124088,3,0.854545,...,2,2,2,2,2,1786,1.000000,0.00,0.666667,0.75
4,0.000007,0.001558,1,0.201023,0.828,0.381679,3,0.014599,2,0.727273,...,1,0,1,1,1,954,0.444444,0.00,0.666667,0.75


In [ ]:
final_x_df.to_pickle('/content/drive/MyDrive/Datasets/dataset/norm_final_x_train.pkl')

In [ ]:
del final_x_df
gc.collect()

759

In [ ]:
categorical_columns = final_test_df.select_dtypes(include='category').columns

numerical_columns = final_test_df.select_dtypes(exclude='category').columns

In [ ]:
final_test_df[numerical_columns] = scaler.fit_transform(final_test_df[numerical_columns])

# Encode categorical columns using LabelEncoder
label_encoder = LabelEncoder()
for column in categorical_columns:
    final_test_df[column] = label_encoder.fit_transform(final_test_df[column])

final_test_df.head()

,TransactionID,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,...,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo,P_emaildomain_bin,P_emaildomain_suffix,R_emaildomain_bin,R_emaildomain_suffix
0,0.000000,0.003109,4,0.540814,0.022,0.378788,3,0.919708,2,0.159091,...,2,2,2,2,2,2226,0.444444,0.0,0.666667,0.75
1,0.000002,0.004769,4,0.188032,0.022,0.378788,3,0.919708,2,0.452273,...,2,2,2,2,2,2226,0.000000,0.0,0.666667,0.75
2,0.000004,0.016645,4,0.199759,0.948,0.378788,3,0.919708,2,0.845455,...,2,2,2,2,2,2226,0.555556,0.0,0.666667,0.75
3,0.000006,0.027744,4,0.574155,0.520,0.378788,3,0.481752,2,0.238636,...,2,2,2,2,2,2226,0.444444,0.0,0.666667,0.75
4,0.000008,0.006612,4,0.978213,0.704,0.378788,2,0.124088,2,0.372727,...,2,2,2,2,2,2226,0.444444,0.0,0.666667,0.75


In [ ]:
final_test_df.to_pickle('/content/drive/MyDrive/Datasets/dataset/norm_final_test.pkl')

In [ ]:
del categorical_columns
del numerical_columns

In [ ]:
gc.collect()

10168

In [ ]:
del final_test_df
gc.collect()

0

## Loading final preprocessed data

In [ ]:
final_y_df = pd.read_pickle("/content/drive/MyDrive/Datasets/dataset/final_y_train.pkl")

In [ ]:
df_x = pd.read_pickle("/content/drive/MyDrive/Datasets/dataset/norm_final_x_train.pkl")
df_y = final_y_df
final_test_df = pd.read_pickle("/content/drive/MyDrive/Datasets/dataset/norm_final_test.pkl")

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_x, df_y, test_size=0.2, random_state=42)

In [ ]:
del df_x
del df_y

In [ ]:
gc.collect()

4023